# 파이토치(PyTorch)로 구현하는 LSTM 언어 모델

본 노트북은 **"Long Short-Term Memory Recurrent Neural Network Architectures for Large Scale Acoustic Modeling" (Sak et al., 2014)** 논문을 기반으로 LSTM(Long Short-Term Memory) 네트워크를 구현합니다.

LSTM은 기존 RNN의 장기 의존성(Long-term Dependency) 문제를 해결하기 위해 설계된 특수한 RNN 아키텍처입니다.

---

## 목차

1. [LSTM 이론 및 구조](#1-lstm-이론-및-구조)
2. [환경 설정](#2-환경-설정)
3. [데이터 로드 및 전처리](#3-데이터-로드-및-전처리)
4. [LSTM 모델 구현](#4-lstm-모델-구현)
5. [학습](#5-학습-training)
6. [시각화 및 결과 분석](#6-결과-시각화)
7. [텍스트 생성](#7-텍스트-생성)

## 1. LSTM 이론 및 구조

### 1.1 RNN의 한계: 기울기 소실 문제

기존 RNN은 시퀀스가 길어질수록 **기울기 소실(Vanishing Gradient)** 또는 **기울기 폭발(Exploding Gradient)** 문제가 발생합니다. 이로 인해 먼 과거의 정보를 현재 시점으로 전달하기 어렵습니다.

### 1.2 LSTM의 핵심 아이디어

LSTM은 **메모리 셀(Memory Cell)**과 **게이트(Gate)** 메커니즘을 도입하여 이 문제를 해결합니다:

```
┌────────────────────────────────────────────────────────────┐
│                     LSTM Memory Block                      │
│                                                            │
│   ┌─────┐    ┌─────┐    ┌─────┐                            │
│   │  f  │    │  i  │    │  o  │    ← 게이트들                │
│   │ 망각 │    │ 입력 │    │ 출력 │                            │
│   └──┬──┘    └──┬──┘    └──┬──┘                            │
│      │          │          │                               │
│      ▼          ▼          ▼                               │
│   ┌──────────────────────────┐                             │
│   │      Memory Cell (c)     │ ← 장기 메모리                  │
│   └──────────────────────────┘                             │
│                  │                                         │
│                  ▼                                         │
│            [Hidden State h]    ← 단기 메모리/출력              │
└────────────────────────────────────────────────────────────┘
```

### 1.3 LSTM 게이트 상세 설명

#### 망각 게이트 (Forget Gate)
- **역할**: 이전 셀 상태에서 어떤 정보를 "잊을지" 결정
- **수식**: $f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$
- **출력**: 0~1 사이 값 (0: 완전히 잊음, 1: 완전히 유지)

#### 입력 게이트 (Input Gate)
- **역할**: 새로운 정보 중 어떤 것을 셀 상태에 "저장할지" 결정
- **수식**: $i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$
- **후보 값**: $\tilde{c}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)$

#### 셀 상태 업데이트 (Cell State Update)
- **역할**: 이전 정보와 새 정보를 조합하여 장기 메모리 갱신
- **수식**: $c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$

#### 출력 게이트 (Output Gate)
- **역할**: 셀 상태에서 어떤 정보를 "출력할지" 결정
- **수식**: $o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$
- **은닉 상태**: $h_t = o_t \odot \tanh(c_t)$

### 1.4 왜 LSTM이 장기 의존성을 학습할 수 있는가?

1. **셀 상태의 직접적인 경로**: $c_t = f_t \odot c_{t-1} + ...$에서 $f_t \approx 1$이면 기울기가 거의 그대로 전파
2. **덧셈 연결**: 곱셈이 아닌 덧셈으로 정보 결합 → 기울기 소실 완화
3. **게이트 메커니즘**: 적응적으로 정보 흐름 제어

## 2. 환경 설정

In [ ]:
import time
from collections import Counter
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
import matplotlib.pyplot as plt

# 재현 가능성을 위한 시드 고정
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Device 설정
def get_device() -> torch.device:
    """사용 가능한 최적의 디바이스 반환"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = get_device()
print(f"사용 장치(Device): {device}")
print(f"PyTorch 버전: {torch.__version__}")

In [ ]:
class Config:
    """모델 및 학습 하이퍼파라미터 설정"""
    
    # 데이터 설정
    dataset_name: str = "Salesforce/wikitext"
    dataset_config: str = "wikitext-2-raw-v1"
    min_count: int = 5  # 최소 출현 빈도
    
    # 모델 하이퍼파라미터
    embed_size: int = 256      # 임베딩 차원
    hidden_size: int = 256     # LSTM 은닉 상태 차원
    num_layers: int = 2        # LSTM 레이어 수
    dropout: float = 0.3       # 드롭아웃 비율
    
    # 학습 설정
    batch_size: int = 20
    time_size: int = 35        # Truncated BPTT 길이
    max_epoch: int = 10        # 충분한 학습
    learning_rate: float = 0.001  # Adam 학습률
    clip_grad: float = 5.0     # Gradient Clipping
    
    def __repr__(self):
        return (
            f"Config(embed_size={self.embed_size}, hidden_size={self.hidden_size}, "
            f"num_layers={self.num_layers}, dropout={self.dropout}, "
            f"batch_size={self.batch_size}, time_size={self.time_size}, "
            f"max_epoch={self.max_epoch}, lr={self.learning_rate})"
        )

config = Config()
print(config)

## 3. 데이터 로드 및 전처리

**Wikitext-2** 데이터셋을 사용합니다. RNN 노트북과 동일한 전처리 과정을 거칩니다.

In [ ]:
print("데이터셋 다운로드 및 로드 중...")
ds = load_dataset(config.dataset_name, config.dataset_config)
print(f"데이터셋 로드 완료!")
print(f"   - Train: {len(ds['train'])} 문장")
print(f"   - Validation: {len(ds['validation'])} 문장")
print(f"   - Test: {len(ds['test'])} 문장")

In [ ]:
def build_vocab(
    dataset, 
    splits: List[str] = ['train', 'validation', 'test'],
    min_count: int = 5
) -> Tuple[Dict[str, int], Dict[int, str]]:
    """
    데이터셋에서 단어 빈도를 카운팅하고 어휘 사전을 구축합니다.
    """
    counter = Counter()
    
    for split in splits:
        for line in dataset[split]['text']:
            words = line.strip().lower().split()
            counter.update(words)
    
    # <unk> 토큰은 항상 0번 ID
    word_to_id = {'<unk>': 0}
    id_to_word = {0: '<unk>'}
    
    # 빈도순으로 정렬하여 ID 부여
    sorted_words = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    valid_words = [word for word, count in sorted_words if count >= min_count]
    
    for word in valid_words:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
    
    return word_to_id, id_to_word

# 어휘 사전 구축
word_to_id, id_to_word = build_vocab(ds, min_count=config.min_count)
vocab_size = len(word_to_id)

print(f"\n어휘 사전 통계:")
print(f"   - 전체 어휘 크기: {vocab_size:,}")
print(f"   - 최소 출현 빈도: {config.min_count}")

In [ ]:
def convert_to_ids(
    dataset, 
    split: str, 
    word_to_id: Dict[str, int]
) -> np.ndarray:
    """
    텍스트 데이터를 단어 ID 시퀀스로 변환합니다.
    """
    ids = []
    unk_id = word_to_id['<unk>']
    
    for line in dataset[split]['text']:
        words = line.strip().lower().split()
        if not words:
            continue
        ids.extend([word_to_id.get(w, unk_id) for w in words])
    
    return np.array(ids, dtype=np.int64)

# 각 분할에 대해 ID 변환
corpus_train = convert_to_ids(ds, 'train', word_to_id)
corpus_valid = convert_to_ids(ds, 'validation', word_to_id)
corpus_test = convert_to_ids(ds, 'test', word_to_id)

print(f"\n코퍼스 크기:")
print(f"   - Train: {len(corpus_train):,} tokens")
print(f"   - Validation: {len(corpus_valid):,} tokens")
print(f"   - Test: {len(corpus_test):,} tokens")

## 4. LSTM 모델 구현

논문의 수식을 직접 구현한 **커스텀 LSTM 셀**과, PyTorch의 `nn.LSTM`을 활용한 언어 모델을 구현합니다.

### 4.1 커스텀 LSTM 셀 (교육용)

LSTM의 내부 동작을 이해하기 위해 직접 구현합니다.

In [ ]:
class LSTMCell(nn.Module):
    """
    커스텀 LSTM 셀 구현 (Peephole Connections 포함).
    
    Hochreiter & Schmidhuber 1997 논문의 수식을 충실히 구현:
        i_t = σ(W_ix * x_t + W_ih * h_{t-1} + W_ci * c_{t-1} + b_i)  # 입력 게이트
        f_t = σ(W_fx * x_t + W_fh * h_{t-1} + W_cf * c_{t-1} + b_f)  # 망각 게이트  
        g_t = tanh(W_gx * x_t + W_gh * h_{t-1} + b_g)  # 후보 셀 상태
        c_t = f_t ⊙ c_{t-1} + i_t ⊙ g_t  # 셀 상태
        o_t = σ(W_ox * x_t + W_oh * h_{t-1} + W_co * c_t + b_o)  # 출력 게이트
        h_t = o_t ⊙ tanh(c_t)  # 은닉 상태
    
    Peephole connections: 셀 상태(c)가 게이트들에 직접 연결되어
    더 정밀한 정보 흐름 제어가 가능합니다.
    
    Args:
        input_size: 입력 차원
        hidden_size: 은닉 상태 차원
    """
    
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # 4개 게이트를 하나의 행렬로 통합 (효율성)
        # 순서: [input_gate, forget_gate, cell_gate, output_gate]
        self.W_x = nn.Linear(input_size, 4 * hidden_size, bias=False)
        self.W_h = nn.Linear(hidden_size, 4 * hidden_size, bias=True)
        
        # Peephole 가중치 (대각 행렬을 벡터로 표현 - 메모리 효율적)
        self.W_ci = nn.Parameter(torch.zeros(hidden_size))  # 입력 게이트용
        self.W_cf = nn.Parameter(torch.zeros(hidden_size))  # 망각 게이트용
        self.W_co = nn.Parameter(torch.zeros(hidden_size))  # 출력 게이트용
        
        # 가중치 초기화
        self._init_weights()
        
    def _init_weights(self):
        """개선된 가중치 초기화"""
        for name, param in self.named_parameters():
            if 'W_x' in name:
                nn.init.xavier_uniform_(param)
            elif 'W_h' in name and 'weight' in name:
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
                # 망각 게이트 편향은 1로 초기화 (초기에 정보 유지 유도)
                n = param.size(0)
                param.data[n//4:n//2].fill_(1.0)
        
        # Peephole 가중치는 작은 값으로 초기화
        nn.init.uniform_(self.W_ci, -0.1, 0.1)
        nn.init.uniform_(self.W_cf, -0.1, 0.1)
        nn.init.uniform_(self.W_co, -0.1, 0.1)
    
    def forward(
        self, 
        x: torch.Tensor, 
        hx: Optional[Tuple[torch.Tensor, torch.Tensor]] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        순전파 계산.
        
        Args:
            x: 입력 (batch, input_size)
            hx: (h_prev, c_prev) 튜플, None이면 0으로 초기화
            
        Returns:
            h_next: 새로운 은닉 상태 (batch, hidden_size)
            c_next: 새로운 셀 상태 (batch, hidden_size)
        """
        batch_size = x.size(0)
        
        # 초기 상태가 없으면 0으로 초기화
        if hx is None:
            h_prev = torch.zeros(batch_size, self.hidden_size, device=x.device)
            c_prev = torch.zeros(batch_size, self.hidden_size, device=x.device)
        else:
            h_prev, c_prev = hx
        
        # 4개 게이트 한번에 계산 (기본 연산)
        gates = self.W_x(x) + self.W_h(h_prev)  # (batch, 4*hidden)
        
        # 게이트 분리
        H = self.hidden_size
        i_gate_pre = gates[:, :H]
        f_gate_pre = gates[:, H:2*H]
        g_gate_pre = gates[:, 2*H:3*H]
        o_gate_pre = gates[:, 3*H:]
        
        # Peephole connections 적용 (c_{t-1}이 입력/망각 게이트에 영향)
        i_gate = torch.sigmoid(i_gate_pre + self.W_ci * c_prev)  # 입력 게이트
        f_gate = torch.sigmoid(f_gate_pre + self.W_cf * c_prev)  # 망각 게이트
        g_gate = torch.tanh(g_gate_pre)  # 후보 셀 상태
        
        # 셀 상태 업데이트
        c_next = f_gate * c_prev + i_gate * g_gate
        
        # 출력 게이트 (c_t가 영향) - Peephole
        o_gate = torch.sigmoid(o_gate_pre + self.W_co * c_next)
        
        # 은닉 상태 계산
        h_next = o_gate * torch.tanh(c_next)
        
        return h_next, c_next

In [ ]:
class LSTMLayer(nn.Module):
    """
    커스텀 LSTM 레이어 - 시퀀스 전체를 처리.
    
    Args:
        input_size: 입력 차원
        hidden_size: 은닉 상태 차원
    """
    
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell = LSTMCell(input_size, hidden_size)
        
    def forward(
        self, 
        x: torch.Tensor, 
        hx: Optional[Tuple[torch.Tensor, torch.Tensor]] = None
    ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        순전파 계산.
        
        Args:
            x: 입력 시퀀스 (batch, time, input_size)
            hx: 초기 (h, c) 상태, None이면 0으로 초기화
            
        Returns:
            outputs: 모든 시간 단계의 은닉 상태 (batch, time, hidden_size)
            (h_n, c_n): 마지막 은닉/셀 상태
        """
        batch_size, time_size, _ = x.shape
        
        # 초기 상태 설정
        if hx is None:
            h = torch.zeros(batch_size, self.hidden_size, device=x.device)
            c = torch.zeros(batch_size, self.hidden_size, device=x.device)
        else:
            h, c = hx
        
        outputs = []
        for t in range(time_size):
            h, c = self.cell(x[:, t, :], (h, c))
            outputs.append(h.unsqueeze(1))
        
        outputs = torch.cat(outputs, dim=1)  # (batch, time, hidden)
        return outputs, (h, c)

### 4.2 LSTM 언어 모델

실제 학습에는 PyTorch의 최적화된 `nn.LSTM`을 사용합니다.

In [ ]:
class LSTMLanguageModel(nn.Module):
    """
    LSTM 기반 언어 모델.
    
    구조: Embedding → Dropout → LSTM → Dropout → Linear
    
    Hochreiter & Schmidhuber 1997 논문 기반 구현.
    Weight Tying 및 개선된 가중치 초기화 적용.
    
    Args:
        vocab_size: 어휘 크기
        embed_size: 임베딩 차원
        hidden_size: LSTM 은닉 상태 차원
        num_layers: LSTM 레이어 수
        dropout: 드롭아웃 비율
        tie_weights: 임베딩-출력 가중치 공유 여부
    """
    
    def __init__(
        self, 
        vocab_size: int, 
        embed_size: int, 
        hidden_size: int,
        num_layers: int = 2,
        dropout: float = 0.2,
        tie_weights: bool = True
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 1. 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # 2. 드롭아웃
        self.drop = nn.Dropout(dropout)
        
        # 3. LSTM 레이어 (다층)
        self.lstm = nn.LSTM(
            input_size=embed_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        # 4. 출력 레이어
        self.fc = nn.Linear(hidden_size, vocab_size)
        
        # Weight Tying: 임베딩과 출력층 가중치 공유
        if tie_weights and embed_size == hidden_size:
            self.fc.weight = self.embedding.weight
        
        # 개선된 가중치 초기화
        self._init_weights()
        
    def _init_weights(self, init_range: float = 0.1):
        """LSTM에 최적화된 가중치 초기화"""
        # 임베딩 초기화
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
        
        # LSTM 가중치 초기화
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                # 입력-은닉 가중치: Xavier 초기화
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:
                # 은닉-은닉 가중치: 직교 초기화
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
                # 망각 게이트 편향을 1로 초기화 (정보 유지 장려)
                n = param.size(0)
                param.data[n//4:n//2].fill_(1.0)
    
    def forward(
        self, 
        x: torch.Tensor, 
        hidden: Optional[Tuple[torch.Tensor, torch.Tensor]] = None
    ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        순전파.
        
        Args:
            x: 입력 단어 ID (batch, time_size)
            hidden: LSTM 초기 상태 (h, c)
            
        Returns:
            output: 로짓 (batch, time_size, vocab_size)
            hidden: 새로운 LSTM 상태
        """
        # 임베딩
        embeds = self.drop(self.embedding(x))  # (batch, time, embed)
        
        # LSTM
        lstm_out, hidden = self.lstm(embeds, hidden)  # (batch, time, hidden)
        
        # 출력층
        output = self.drop(lstm_out)
        output = self.fc(output)  # (batch, time, vocab)
        
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """은닉 상태 초기화"""
        weight = next(self.parameters())
        h = weight.new_zeros(self.num_layers, batch_size, self.hidden_size)
        c = weight.new_zeros(self.num_layers, batch_size, self.hidden_size)
        return (h, c)
    
    def get_num_params(self) -> int:
        """학습 가능한 파라미터 수 반환"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [ ]:
# 모델 생성
model = LSTMLanguageModel(
    vocab_size=vocab_size,
    embed_size=config.embed_size,
    hidden_size=config.hidden_size,
    num_layers=config.num_layers,
    dropout=config.dropout,
    tie_weights=True
).to(device)

print("모델 구조:")
print(model)
print(f"\n총 파라미터 수: {model.get_num_params():,}")

## 5. 학습 (Training)

### Truncated BPTT
RNN/LSTM의 역전파는 시간을 거슬러 올라가며 수행됩니다. 전체 시퀀스에 대해 BPTT를 수행하면 비용이 크므로 일정 길이로 잘라서 수행합니다.

In [ ]:
def get_batch(
    corpus: np.ndarray,
    batch_size: int,
    time_size: int,
    time_idx: int,
    offsets: List[int],
    data_size: int
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    학습용 미니배치 생성.
    """
    batch_x, batch_t = [], []
    
    for i in range(batch_size):
        ptr = (offsets[i] + time_idx) % data_size
        indices = [(ptr + t) % data_size for t in range(time_size + 1)]
        raw_data = corpus[indices]
        batch_x.append(raw_data[:-1])
        batch_t.append(raw_data[1:])
    
    return (
        torch.tensor(np.array(batch_x), dtype=torch.long),
        torch.tensor(np.array(batch_t), dtype=torch.long)
    )

In [ ]:
def evaluate(
    model: nn.Module,
    corpus: np.ndarray,
    batch_size: int,
    time_size: int,
    criterion: nn.Module
) -> float:
    """모델 평가하고 평균 손실 반환"""
    model.eval()
    data_size = len(corpus)
    max_iters = data_size // (batch_size * time_size)
    jump = (data_size - 1) // batch_size
    offsets = [i * jump for i in range(batch_size)]
    
    total_loss = 0.0
    hidden = None
    
    with torch.no_grad():
        for iter_idx in range(max_iters):
            inputs, targets = get_batch(
                corpus, batch_size, time_size, 
                iter_idx * time_size, offsets, data_size
            )
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs, hidden = model(inputs, hidden)
            loss = criterion(
                outputs.reshape(-1, model.vocab_size),
                targets.reshape(-1)
            )
            total_loss += loss.item()
            
            if hidden is not None:
                hidden = tuple(h.detach() for h in hidden)
    
    return total_loss / max_iters if max_iters > 0 else float('inf')

In [ ]:
# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2
)

# 배치 샘플링을 위한 오프셋 계산
data_size = len(corpus_train)
max_iters = data_size // (config.batch_size * config.time_size)
jump = (data_size - 1) // config.batch_size
offsets = [i * jump for i in range(config.batch_size)]

# 학습 기록
history = {
    'train_loss': [],
    'train_ppl': [],
    'valid_loss': [],
    'valid_ppl': []
}

print(f"학습 시작!")
print(f"   - 에폭 수: {config.max_epoch}")
print(f"   - 배치 크기: {config.batch_size}")
print(f"   - 이터레이션/에폭: {max_iters:,}")
print(f"   - Optimizer: Adam (lr={config.learning_rate})")
print("=" * 80)

In [ ]:
start_time = time.time()
best_val_loss = float('inf')
best_val_ppl = float('inf')

for epoch in range(config.max_epoch):
    model.train()
    hidden = None
    total_loss = 0.0
    
    for iter_idx in range(max_iters):
        # 미니배치 생성
        time_idx = iter_idx * config.time_size
        inputs, targets = get_batch(
            corpus_train, config.batch_size, config.time_size,
            time_idx, offsets, data_size
        )
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Truncated BPTT: 은닉 상태 분리
        if hidden is not None:
            hidden = tuple(h.detach() for h in hidden)
        
        # 순전파
        outputs, hidden = model(inputs, hidden)
        loss = criterion(
            outputs.reshape(-1, vocab_size),
            targets.reshape(-1)
        )
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_grad)
        
        optimizer.step()
        total_loss += loss.item()
    
    # 에폭 통계
    train_loss = total_loss / max_iters
    train_ppl = np.exp(train_loss)
    
    # 검증
    valid_loss = evaluate(model, corpus_valid, config.batch_size, config.time_size, criterion)
    valid_ppl = np.exp(valid_loss)
    
    # 학습률 스케줄러 업데이트
    scheduler.step(valid_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # 기록
    history['train_loss'].append(train_loss)
    history['train_ppl'].append(train_ppl)
    history['valid_loss'].append(valid_loss)
    history['valid_ppl'].append(valid_ppl)
    
    elapsed = time.time() - start_time
    print(
        f"| Epoch {epoch+1:2d}/{config.max_epoch} "
        f"| Train PPL {train_ppl:8.2f} "
        f"| Valid PPL {valid_ppl:8.2f} "
        f"| LR {current_lr:.6f} | Time {elapsed:5.0f}s |"
    )
    
    # 최적 모델 저장
    if valid_ppl < best_val_ppl:
        best_val_ppl = valid_ppl
        best_val_loss = valid_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'epoch': epoch + 1,
            'best_val_ppl': best_val_ppl,
            'config': {
                'vocab_size': vocab_size,
                'embed_size': config.embed_size,
                'hidden_size': config.hidden_size,
                'num_layers': config.num_layers
            }
        }, 'best_lstm_lm.pth')

print("=" * 80)
print(f"학습 완료! 총 소요 시간: {time.time() - start_time:.1f}초")
print(f"최종 Train PPL: {train_ppl:.2f}")
print(f"최종 Valid PPL: {valid_ppl:.2f}")
print(f"최적 Valid PPL: {best_val_ppl:.2f}")

# 최적 모델 로드
checkpoint = torch.load('best_lstm_lm.pth', weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
print("최적 모델 로드 완료!")

In [ ]:
# 모델 저장
model_path = 'lstm_lm.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'config': {
        'vocab_size': vocab_size,
        'embed_size': config.embed_size,
        'hidden_size': config.hidden_size,
        'num_layers': config.num_layers
    },
    'history': history
}, model_path)
print(f"모델 저장 완료: {model_path}")

## 6. 결과 시각화

학습 과정에서의 손실(Loss)과 Perplexity(PPL) 변화를 시각화합니다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss 그래프
ax1 = axes[0]
epochs = range(1, len(history['train_loss']) + 1)
ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2)
ax1.plot(epochs, history['valid_loss'], 'r-s', label='Valid Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# PPL 그래프
ax2 = axes[1]
ax2.plot(epochs, history['train_ppl'], 'b-o', label='Train PPL', linewidth=2)
ax2.plot(epochs, history['valid_ppl'], 'r-s', label='Valid PPL', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Perplexity', fontsize=12)
ax2.set_title('Training & Validation Perplexity', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n최종 결과:")
print(f"   - Train PPL: {history['train_ppl'][-1]:.2f}")
print(f"   - Valid PPL: {history['valid_ppl'][-1]:.2f}")

## 7. 텍스트 생성

학습된 LSTM 모델을 사용하여 텍스트를 생성합니다.

In [ ]:
def generate_text(
    model: nn.Module,
    start_word: str,
    word_to_id: Dict[str, int],
    id_to_word: Dict[int, str],
    length: int = 50,
    temperature: float = 1.0
) -> str:
    """
    학습된 모델로 텍스트를 생성합니다.
    
    Args:
        model: 학습된 LSTM 모델
        start_word: 시작 단어
        word_to_id: 단어 → ID 매핑
        id_to_word: ID → 단어 매핑
        length: 생성할 단어 수
        temperature: 샘플링 온도 (높을수록 다양함)
        
    Returns:
        생성된 텍스트
    """
    model.eval()
    
    if start_word.lower() not in word_to_id:
        return f"'{start_word}'는 어휘에 없는 단어입니다."
    
    input_id = torch.tensor([[word_to_id[start_word.lower()]]], device=device)
    hidden = None
    result = [start_word.lower()]
    
    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_id, hidden)
            
            logits = output.squeeze() / temperature
            probs = torch.softmax(logits, dim=0)
            
            next_id = torch.multinomial(probs, 1).item()
            result.append(id_to_word[next_id])
            
            input_id = torch.tensor([[next_id]], device=device)
    
    return ' '.join(result)


# 텍스트 생성 예시
print("\n생성된 텍스트:\n")
print("="*60)

for start_word in ['the', 'in', 'he', 'it']:
    if start_word in word_to_id:
        generated = generate_text(
            model, start_word, word_to_id, id_to_word,
            length=30, temperature=0.8
        )
        print(f"\n[{start_word}]로 시작:")
        print(f"  {generated}")

print("\n" + "="*60)

In [ ]:
# Temperature에 따른 생성 결과 비교
print("\nTemperature에 따른 생성 결과 비교:\n")

start = 'the'
for temp in [0.5, 0.8, 1.0, 1.2]:
    text = generate_text(
        model, start, word_to_id, id_to_word,
        length=20, temperature=temp
    )
    print(f"Temperature={temp}: {text}")
    print()